### Are we in SWAN?

In [1]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack/for_martin

[Errno 2] No such file or directory: '/eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack/for_martin'
/home/martin/owncloud/work/cern/presentations/20200619_be_seminar_sixtracklib/carlo_emilio


In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user sixtrackwrap
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

In [1]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

# Taking advantage of the step-by-step tracking

SixTrackLib allows the creation of track jobs properly allocated on the GPU device. With these jobs, we can efficiently gather the characteristics of every particle turn after turn efficiently without loosing the GPU parallel capabilites, as the data can be gathered from the GPU directly with an optimized memory access.

Thanks to that, we were able to implement some first full-track analysis of particle transversal dynamics, for inspecting the presence of resonances for specific initial conditions.

In this notebook, we show just the most "colorful" plots we have made in our preliminary analysis.

## Backend setup and libraries

In [2]:
%matplotlib widget

In [3]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

# Personal libraries
#import sixtrackwrap_light as sx
import sixtrackwrap as sx

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

from scipy.special import lambertw

## Some step-by-step trackings

Here we have performed in a separate instance a radial scan of various angular coordinates. For each angle, given a value $N_\text{max}$ of turns, we look for the last stable particle after $N_\text{max}$ turns.

Once we have found it, we re-track the particle while saving its whole transversal path. We then analyze its motion characteristics in the 4D space considering normalized polar coordinates $(r, \alpha, \theta_1, \theta_2)$.

In this specific plot, we analyze how the particle "explores" the $(\theta_1, \theta_2)$ space by considering a 2D binning of the $N_\text{max}$ steps the particle takes before becoming lost.

In the left plot, we show for each bin the average radial distance the particle has for that specific $(\theta_1, \theta_2)$ interval. The white bins indicates a NaN value, meaning that the particle has not visited that specific interval.

In the right plot, we show for each bin the number of times the particle has visited that specific $(\theta_1, \theta_2)$ interval.

With the slider, you can select the $\alpha_0$ angle as starting considition. In this setup, both $\theta$ angles are set to 0 as starting condition.

In [4]:
with open("data/matrices_2.pkl", 'rb') as f:
    count_matrix, avg_matrix = pickle.load(f)
samples = 2049
n_subdivisions = 128
max_turns = 10000

In [5]:
count_total, average_total, result_total, validity_total = sx.recursive_accumulation(count_matrix, avg_matrix)
alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2

In [6]:
fig2 = plt.figure()

def update_scheme(j):
    fig2.clear()
    axs2 = fig2.subplots(1, 2)
    i = 0
    j = len(count_total[0]) - 1 - j
    coso = axs2[0].imshow(average_total[i][j], origin="lower", extent=(0, np.pi*2, 0, np.pi*2))
    axs2[0].set_title("Average radius measured\n$\\alpha = {:.4}\pi$".format(alpha_values[j]/np.pi))
    axs2[0].set_xlabel("$\\theta_1$")
    axs2[0].set_ylabel("$\\theta_2$")
    cb = fig2.colorbar(coso, ax=axs2[0])
    #cb.ax.plot([0, 100], [DA_2[(samples, 'refined', 'mc')][0]]*2, 'w')

    axs2[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
    axs2[0].xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))
    axs2[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
    axs2[0].yaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))

    coso = axs2[1].imshow(count_total[i][j], origin="lower", extent=(0, np.pi*2, 0, np.pi*2), vmin=0)
    axs2[1].set_title("Number of samples\n$\\alpha = {:.4}\pi$".format(alpha_values[j]/np.pi))
    axs2[1].set_xlabel("$\\theta_1$")
    axs2[1].set_ylabel("$\\theta_2$")
    fig2.colorbar(coso, ax=axs2[1])

    axs2[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
    axs2[1].xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))
    axs2[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
    axs2[1].yaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))
    
    fig2.suptitle("2D binning $(128\\times 128)$ over the $(\\theta_1, \\theta_2)$ space of a particle tracked for 10000 turns.")

k=widgets.IntSlider(value=0, min=0, max=len(count_total[0]) - 1, step=1)
ui2 = widgets.VBox([
    widgets.Label("Sample index to visualize"), k,
])
    
out2 = widgets.interactive_output(
    update_scheme,
    {"j":k}
)

display(ui2, out2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

As you can see from the plot above, different $\alpha$ starting conditions implies very different behaviours for the particle transversal dynamics. And it's in our interest to inspect the charatcerstics of these resonances.

A first qualitative measurement of these resonance behaviours is to evaluate the percentage of empty bins in the left plots above: more empty plots implies less uniform diffusion in the $(\theta_1, \theta_2)$ space and, therefore, stronger resonace effects.

We plot this measure down here.

In [7]:
count_total, average_total, result_total, validity_total = sx.recursive_accumulation(count_matrix, avg_matrix)
alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2

nan_data = []

for j in range(0, len(count_total[2])):
    nan_data.append(np.count_nonzero(np.isnan(average_total[2][j])) / ((n_subdivisions / 4) ** 2))

nan_data = np.asarray(nan_data)
x = np.cos(alpha_values[::4]) * nan_data[::4]
y = np.sin(alpha_values[::4]) * nan_data[::4]
x = np.append(x, [0.0])
y = np.append(y, [0.0])

#plt.fill(x, y)
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(alpha_values, nan_data, linewidth=0.2, c="C0", alpha=0.5)
ax.scatter(alpha_values, nan_data, s=0.75)
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "{:2}".format(x/np.pi) + "$\\ \\pi$"))
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi/8))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "{}".format(int(x * 100)) + "$\\%$"))
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.set_xlabel("$\\alpha$ angle")
ax.set_ylabel("Percentage of empty bins")
ax.set_title("Percentage of empty bins for different intial $\\alpha$ angles. $N$ bins $= ({}\\times{}) = {}$, $N$ turns $= {}$\n(Higher percentage implies less `diffusion')".format(n_subdivisions // 4, n_subdivisions // 4, (n_subdivisions // 4) ** 2, max_turns))
ax.set_xlim(0, np.pi/2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 1.5707963267948966)

These analysis are made possible by the fact that SixTrackLib makes the execution of single parallelized tracking steps extremely easy and well optimized.